In [ ]:
try:
    from openmdao.utils.notebook_utils import notebook_mode
except ImportError:
    !python -m pip install openmdao[notebooks]

In [ ]:
from openmdao.utils.assert_utils import assert_near_equal

import os
if os.path.exists('cases.sql'):
    os.remove('cases.sql')

# Getting Data from a Case

The `Case` object contains all the information about a specific recorded case whether it was recorded by
a problem, driver, system, or solver. `Case` objects have a number methods for accessing variables and their values.

## Example of Getting Variable Data from Case Recording of a Driver

Here are the methods typically used when retrieving data from the recording of a :code:`Driver`.

```{eval-rst}
.. automethod:: openmdao.recorders.case.Case.get_objectives
    :noindex:

.. automethod:: openmdao.recorders.case.Case.get_constraints
    :noindex:

.. automethod:: openmdao.recorders.case.Case.get_design_vars
    :noindex:

.. automethod:: openmdao.recorders.case.Case.get_responses
    :noindex:
```

The following example shows how to use these methods to easily check the variables of interest
from the first driver iteration.

In [ ]:
from openmdao.utils.notebook_utils import get_code
from myst_nb import glue
glue("code_src80", get_code("openmdao.test_suite.components.sellar_feature.SellarDerivatives"), display=False)

:::{Admonition} `SellarDerivatives` class definition 
:class: dropdown

{glue:}`code_src80`
:::

In [ ]:
import openmdao.api as om
from openmdao.test_suite.components.sellar_feature import SellarDerivatives

import numpy as np

prob = om.Problem(model=SellarDerivatives())

model = prob.model
model.add_design_var('z', lower=np.array([-10.0, 0.0]),
                          upper=np.array([10.0, 10.0]))
model.add_design_var('x', lower=0.0, upper=10.0)
model.add_objective('obj')
model.add_constraint('con1', upper=0.0)
model.add_constraint('con2', upper=0.0)

model.nonlinear_solver = om.NonlinearBlockGS()
model.linear_solver = om.ScipyKrylov()

driver = prob.driver = om.ScipyOptimizeDriver(optimizer='SLSQP', tol=1e-9, disp=False)

driver.add_recorder(om.SqliteRecorder("cases.sql"))

driver.recording_options['includes'] = []
driver.recording_options['record_objectives'] = True
driver.recording_options['record_constraints'] = True
driver.recording_options['record_desvars'] = True

prob.setup()
prob.set_solver_print(0)
prob.run_driver()
prob.cleanup()

cr = om.CaseReader("cases.sql")
driver_cases = cr.list_cases('driver')

In [ ]:
assert driver_cases == [ f"rank0:ScipyOptimize_SLSQP|{n}" for n in range(driver.iter_count)]

In [ ]:
case = cr.get_case(driver_cases[0])
case.outputs.keys()

In [ ]:
assert sorted(case.outputs.keys()) == ['con1', 'con2', 'obj', 'x', 'z']

In [ ]:
objs = case.get_objectives()
cons = case.get_constraints()
dvs = case.get_design_vars()
rsps = case.get_responses()

Using `keys()` will give you the promoted variable names:

In [ ]:
print((sorted(objs.keys()), sorted(cons.keys()), sorted(dvs.keys())))

In [ ]:
assert sorted(objs.keys()) == ['obj']
assert sorted(cons.keys()) == ['con1', 'con2']
assert sorted(dvs.keys()) == ['x', 'z']

Alternatively, you can get the absolute names:

In [ ]:
print((sorted(objs.absolute_names()), sorted(cons.absolute_names()), sorted(dvs.absolute_names())))

In [ ]:
assert sorted(objs.absolute_names()) == ['obj_cmp.obj']
assert sorted(cons.absolute_names()) == ['con_cmp1.con1', 'con_cmp2.con2']
assert sorted(dvs.absolute_names()) == ['x', 'z']

You can access variable values using either the promoted or the absolute name:

In [ ]:
print('objective (obj):\t', objs['obj'], objs['obj_cmp.obj'])
print('constraint (con1):\t', cons['con1'], cons['con_cmp1.con1'])
# Note that x is supplied by an automatically generated IndepVarComp
print('design vars (x):\t', dvs['x'], dvs['_auto_ivc.v1'])
print('response vars (con2):\t', rsps['con2'], rsps['con_cmp2.con2'])

In [ ]:
assert objs['obj'] == objs['obj_cmp.obj']
assert cons['con1'] == cons['con_cmp1.con1']
assert dvs['x'] == dvs['_auto_ivc.v1']
assert rsps['con2'] == rsps['con_cmp2.con2']

You can also access the variables directly from the case object:

In [ ]:
print((case['obj'], case['obj_cmp.obj']))
print((case['x'], case['_auto_ivc.v1']))

In [ ]:
assert case['obj'] == case['obj_cmp.obj']
assert case['x'] == case['_auto_ivc.v1']

## Getting Variable Data from Case Recording of a Problem

Here are the methods typically used when retrieving data from the recording of a `Problem`.

```{eval-rst}
.. automethod:: openmdao.recorders.case.Case.list_inputs
    :noindex:

.. automethod:: openmdao.recorders.case.Case.list_outputs
    :noindex:
```
The following example shows how to use these methods.

In [ ]:
from openmdao.utils.notebook_utils import get_code
from myst_nb import glue
glue("code_src81", get_code("openmdao.test_suite.components.sellar.SellarProblem"), display=False)

:::{Admonition} `SellarProblem` class definition 
:class: dropdown

{glue:}`code_src81`
:::

In [ ]:
import openmdao.api as om
from openmdao.test_suite.components.sellar import SellarProblem
from openmdao.test_suite.components.sellar_feature import SellarDerivatives

prob = SellarProblem(model_class=SellarDerivatives)

recorder = om.SqliteRecorder("cases.sql")
prob.model.nonlinear_solver = om.NonlinearBlockGS()
prob.model.linear_solver = om.ScipyKrylov()

prob.model.add_recorder(recorder)
prob.model.recording_options['record_residuals'] = True

prob.setup()

d1 = prob.model.d1
d1.add_recorder(recorder)

prob.run_driver()
prob.cleanup()

cr = om.CaseReader("cases.sql")

In [ ]:
system_cases = cr.list_cases('root.d1')

`list_inputs()` and `list_outputs()` will print a report to the screen as well as returning a list of the variables and their values.

In [ ]:
case = cr.get_case(system_cases[1])

case_inputs = case.list_inputs()
case_outputs = case.list_outputs()

In [ ]:
from pprint import pprint
pprint(case_inputs)
pprint(case_outputs)

In [ ]:
assert_near_equal(case_inputs[0][1]['val'], [1.], tolerance=1e-10) # d1.x
assert_near_equal(case_inputs[1][1]['val'], [12.27257053], tolerance=1e-10) # d1.y2
assert_near_equal(case_inputs[2][1]['val'], [5., 2.], tolerance=1e-10) # d1.z
assert_near_equal(case_outputs[0][1]['val'], [25.545485893882876], tolerance=1e-10) # d1.y1

The `list_inputs()` and `list_outputs()` methods have optional arguments that let you filter based on variable names what gets listed. This is shown in the following examples.

In [ ]:
from openmdao.utils.notebook_utils import get_code
from myst_nb import glue
glue("code_src82", get_code("openmdao.core.tests.test_expl_comp.RectangleComp"), display=False)

:::{Admonition} `RectangleComp` class definition 
:class: dropdown

{glue:}`code_src82`
:::

In [ ]:
import openmdao.api as om
from openmdao.core.tests.test_expl_comp import RectangleComp

model = om.Group()
prob = om.Problem(model)
model.add_recorder(om.SqliteRecorder('cases.sql'))

model.add_subsystem('rect', RectangleComp(), promotes=['length', 'width', 'area'])

prob.setup(check=False)

prob.set_val('length', 100.)
prob.set_val('width', 60.0)

prob.run_model()

prob.cleanup()

cr = om.CaseReader("cases.sql")
case = cr.get_case(0)

In [ ]:
# Inputs with includes
inputs = case.list_inputs(includes=['*length'])
print(inputs)

In [ ]:
assert objs['obj'] == objs['obj_cmp.obj']
assert cons['con1'] == cons['con_cmp1.con1']
assert dvs['x'] == dvs['_auto_ivc.v1']
assert rsps['con2'] == rsps['con_cmp2.con2']

In [ ]:
# Inputs with excludes
inputs = case.list_inputs(excludes=['*length'])
print(inputs)

In [ ]:
assert len(inputs) == 1
assert inputs[0][0] == 'rect.width'
assert inputs[0][1]['val'] == 60.

In [ ]:
# Outputs with includes
outputs = case.list_outputs(includes=['*area'])
print(outputs)

In [ ]:
assert len(outputs) == 1
assert outputs[0][0] == 'rect.area'
assert outputs[0][1]['val'] == 6000.

Finally, you can also make use of the variable tagging feature when getting values from cases. This example shows how to do that.

In [ ]:
import openmdao.api as om

class RectangleCompWithTags(om.ExplicitComponent):
    """
    A simple Explicit Component that also has input and output with tags.
    """

    def setup(self):
        self.add_input('length', val=1., tags=["tag1", "tag2"])
        self.add_input('width', val=1., tags=["tag2"])
        self.add_output('area', val=1., tags="tag1")

    def setup_partials(self):
        self.declare_partials('*', '*')

    def compute(self, inputs, outputs):
        outputs['area'] = inputs['length'] * inputs['width']

model = om.Group()
prob = om.Problem(model)
model.add_recorder(om.SqliteRecorder('cases.sql'))

model.add_subsystem('rect', RectangleCompWithTags(), promotes=['length', 'width', 'area'])

prob.setup(check=False)

prob.set_val('length', 100.0)
prob.set_val('width', 60.0)

prob.run_model()

prob.cleanup()

cr = om.CaseReader("cases.sql")
case = cr.get_case(0)

In [ ]:
# Inputs with tag that matches
inputs = case.list_inputs(tags="tag1")
print(inputs)

In [ ]:
assert len(inputs) == 1
assert inputs[0][0] == 'rect.length'
assert inputs[0][1]['val'] == 100.

In [ ]:
# Inputs with multiple tags
inputs = case.list_inputs(tags=["tag1", "tag2"])
print(inputs)

In [ ]:
assert len(inputs) == 2
assert inputs[0][0] == 'rect.length'
assert inputs[0][1]['val'] == 100.
assert inputs[1][0] == 'rect.width'
assert inputs[1][1]['val'] == 60.

In [ ]:
# Outputs with tag that does match
outputs = case.list_outputs(tags="tag1")
print(outputs)

In [ ]:
assert len(outputs) == 1
assert outputs[0][0] == 'rect.area'
assert outputs[0][1]['val'] == 6000.

## Getting Variable Data from Case By Specifying Variable Name and Units Desired

You can also get variable values from a `Case` like you would from a `Problem` using dictionary-like access
or, if you want the value in different units, using the `get_val` method.


```{eval-rst}
.. automethod:: openmdao.recorders.case.Case.get_val
    :noindex:
```

This example shows both methods of getting variable data by name.

In [ ]:
import openmdao.api as om

model = om.Group()
model.add_recorder(om.SqliteRecorder('cases.sql'))

speed = om.ExecComp('v=x/t', x={'units': 'm'}, t={'units': 's'}, v={'units': 'm/s'})

model.add_subsystem('speed', speed, promotes=['x', 't', 'v'])

prob = om.Problem(model)
prob.setup()

prob.set_val('x', 100., units='m')
prob.set_val('t', 60., units='s')

prob.run_model()
prob.cleanup()

cr = om.CaseReader('cases.sql')
case = cr.get_case(0)

In [ ]:
print(case['x'])

In [ ]:
assert case['x'] == 100.

In [ ]:
print(case.get_val('x', units='ft'))

In [ ]:
assert_near_equal(case.get_val('x', units='ft'), 328.0839895, 1e-8)

In [ ]:
print(case['v'])

In [ ]:
assert_near_equal(case['v'], 1.66666667, 1e-8)

In [ ]:
print(case.get_val('v', units='ft/s'))

In [ ]:
assert_near_equal(case.get_val('v', units='ft/s'),5.46806649, 1e-8)

## Getting Derivative Data from a Case

A driver has the ability to record derivatives but it is not enabled by default. If you do enable
this option, the recorded cases will have a value for the `jacobian`.

In [ ]:
from openmdao.utils.notebook_utils import get_code
from myst_nb import glue
glue("code_src83", get_code("openmdao.test_suite.components.sellar_feature.SellarMDA"), display=False)

:::{Admonition} `SellarMDA` class definition 
:class: dropdown

{glue:}`code_src83`
:::

In [ ]:
import openmdao.api as om
from openmdao.test_suite.components.sellar_feature import SellarMDA

import numpy as np

prob = om.Problem(model=SellarMDA())

model = prob.model
model.add_design_var('z', lower=np.array([-10.0, 0.0]),
                          upper=np.array([10.0, 10.0]))
model.add_design_var('x', lower=0.0, upper=10.0)
model.add_objective('obj')
model.add_constraint('con1', upper=0.0)
model.add_constraint('con2', upper=0.0)

driver = prob.driver = om.ScipyOptimizeDriver(optimizer='SLSQP', tol=1e-9, disp=False)
driver.recording_options['record_derivatives'] = True

driver.add_recorder(om.SqliteRecorder('cases.sql'))

prob.setup()
prob.set_solver_print(0)
prob.run_driver()
prob.cleanup()

cr = om.CaseReader('cases.sql')

In [ ]:
# Get derivatives associated with the last iteration.
derivs = cr.get_case(-1).derivatives
derivs

In [ ]:
# Get specific derivative.
print(derivs['obj', 'z'])

In [ ]:
# check that derivatives have been recorded.
assert set(derivs.keys()) == set([
    ('obj', 'z'), ('con2', 'z'), ('con1', 'x'),
    ('obj', 'x'), ('con2', 'x'), ('con1', 'z')
])

# check specific derivative.
assert_near_equal(derivs['obj', 'z'], derivs['obj', 'z'])

Problem recording can also include recording of the derivatives as this example shows.

In [ ]:
import openmdao.api as om
from openmdao.test_suite.components.eggcrate import EggCrate

prob = om.Problem()

model = prob.model
model.add_subsystem('egg_crate', EggCrate(), promotes=['x', 'y', 'f_xy'])
model.add_design_var('x', lower=-50.0, upper=50.0)
model.add_design_var('y', lower=-50.0, upper=50.0)
model.add_objective('f_xy')

prob.driver = om.ScipyOptimizeDriver(disp=False, tol=1e-9)

prob.recording_options['record_derivatives'] = True
prob.add_recorder(om.SqliteRecorder('cases.sql'))

prob.setup()
prob.set_solver_print(0)

prob.set_val('x', 2.5)
prob.set_val('y', 2.5)

prob.run_driver()

case_name_1 = "c1"
prob.record(case_name_1)

In [ ]:
assert_near_equal([prob.get_val('x'), prob.get_val('y'), prob.get_val('f_xy')],
                  [[3.01960159], [3.01960159], [18.97639468]], 1e-6)

In [ ]:
prob.set_val('x', 0.1)
prob.set_val('y', -0.1)

prob.run_driver()

case_name_2 = "c2"
prob.record(case_name_2)

prob.cleanup()

In [ ]:
assert_near_equal([prob.get_val('x'), prob.get_val('y'), prob.get_val('f_xy')],
                  [[-2.14311975e-08], [2.14312031e-08], [2.388341e-14]], 1e-6)

In [ ]:
cr = om.CaseReader('cases.sql')

c1 = cr.get_case(case_name_1)
c2 = cr.get_case(case_name_2)

print(c1.derivatives)
print(c2.derivatives)

In [ ]:
num_problem_cases = len(cr.list_cases('problem'))
assert num_problem_cases == 2

# check that derivatives have been recorded properly.
assert_near_equal(c1.derivatives[('f_xy', 'x')][0], 0.0, 1e-4)
assert_near_equal(c1.derivatives[('f_xy', 'y')][0], 0.0, 1e-4)

assert_near_equal(c2.derivatives[('f_xy', 'x')][0], 0.0, 1e-4)
assert_near_equal(c2.derivatives[('f_xy', 'y')][0], 0.0, 1e-4)

```{note}
For both `Driver` and `Problem`, the recording of the derivatives is not affected by
the `includes` and `excludes` options.
```